In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm 
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score
)

MODEL_NAME = "finiteautomata/bertweet-base-sentiment-analysis"
MODEL_PATH = f"../models/{MODEL_NAME}"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.save_pretrained(MODEL_PATH)

# model = AutoModelForSequenceClassification.from_pretrained(
#     MODEL_PATH,
#     local_files_only=True
# )

/opt/homebrew/Caskroom/miniforge/base/envs/sentiment_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 338/338 [00:00<00:00, 692kB/s]
vocab.txt: 100%|██████████| 843k/843k [00:00<00:00, 12.1MB/s]
bpe.codes: 100%|██████████| 1.08M/1.08M [00:00<00:00, 10.9MB/s]
added_tokens.json: 100%|██████████| 22.0/22.0 [00:00<00:00, 67.7kB/s]
special_tokens_map.json: 100%|██████████| 167/167 [00:00<00:00, 386kB/s]
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
config.json: 100%|██████████| 949/949 [00:00<00:00, 2.65MB/s]
pytorch_model.bin: 100%|██████████| 540M/540M [00:48<00:00, 11.0MB/s] 


In [18]:
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        # t = '@user' if t.startswith('@') and len(t) > 1 else t
        # t = 'http' if t.startswith('http') else t
        t = '' if t.startswith(('@', 'http')) and len(t) > 1 else t
        new_text.append(t)
    return " ".join(new_text)

def infer(text):
    labels = ['negative', 'neutral', 'positive']
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    output = ranking[0]
    label = labels[output]
    return text, label

In [19]:
def evaluate(predictions, true_labels):
    cls_report = classification_report(
        true_labels, 
        predictions
    )
    cnf_matrix = confusion_matrix(
        true_labels, 
        predictions
    )
    accuracy = accuracy_score(true_labels, predictions)

    print("Classification Report:\n", cls_report)
    print("Confusion Matrix:\n", cnf_matrix)
    print("Accuracy: ", accuracy)


def test_batch(csv_path):
    df = pd.read_csv(csv_path)
    data = df['text'].tolist()
    true_labels = df['expected_sentiment'].tolist()
    predictions = []

    for i in tqdm(range(len(data)), desc="Processing"):
        text = data[i]
        true_label = true_labels[i]
        processed_text, predicted_label = infer(text)
        predictions.append(predicted_label)

    evaluate(predictions, true_labels)

In [20]:
csv_path = '../data/sentiment_test_cases.csv'
test_batch(csv_path)

Processing: 100%|██████████| 498/498 [00:20<00:00, 24.42it/s]

Classification Report:
               precision    recall  f1-score   support

    negative       0.92      0.86      0.89       177
     neutral       0.84      0.80      0.82       139
    positive       0.83      0.92      0.87       182

    accuracy                           0.87       498
   macro avg       0.87      0.86      0.86       498
weighted avg       0.87      0.87      0.87       498

Confusion Matrix:
 [[153  11  13]
 [  8 111  20]
 [  5  10 167]]
Accuracy:  0.8654618473895582
